In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

In [ ]:
npi_list = pickle.load(open('./data/filtered_npi.pkl', 'rb'))
npi_list = [str(npi) for npi in npi_list]

In [25]:
data_dict = {}
'''
    Structure of data_dict:
    Key: Trial NCTID
    Value: {
        [
            {
                'NPI': Amount, ...
            },
            {
                'Hospital ID': {NPI: Amount, ...}
            },
            {
                'Entity name': {NPI: Amount, ...}
            }
        ]
'''

for i in range(2017, 2022):
    payment = pd.read_csv('D:/Data/payment/Research/OP_DTL_RSRCH_PGYR' + str(i) + '_P01202023.csv', dtype={'Covered_Recipient_NPI':str, 'Principal_Investigator_1_NPI': str, 'Principal_Investigator_2_NPI': str, 'Principal_Investigator_3_NPI': str, 'Principal_Investigator_4_NPI': str, 'Principal_Investigator_5_NPI': str, 'ClinicalTrials_Gov_Identifier': str})

    payment = payment[(payment['Total_Amount_of_Payment_USDollars'] > 0) & payment['ClinicalTrials_Gov_Identifier'].notna()]
    institution_payment = payment[(payment['Principal_Investigator_1_NPI'].isin(npi_list) | payment['Principal_Investigator_2_NPI'].isin(npi_list) | payment['Principal_Investigator_3_NPI'].isin(npi_list) | payment['Principal_Investigator_4_NPI'].isin(npi_list) | payment['Principal_Investigator_5_NPI'].isin(npi_list)) & payment['Teaching_Hospital_ID'].notnull() & payment['Noncovered_Recipient_Entity_Name'].isnull()]
    entity_payment = payment[(payment['Principal_Investigator_1_NPI'].isin(npi_list) | payment['Principal_Investigator_2_NPI'].isin(npi_list) | payment['Principal_Investigator_3_NPI'].isin(npi_list) | payment['Principal_Investigator_4_NPI'].isin(npi_list) | payment['Principal_Investigator_5_NPI'].isin(npi_list)) & payment['Noncovered_Recipient_Entity_Name'].notnull() & payment['Teaching_Hospital_ID'].isnull()]
    individual_payment = payment[payment['Covered_Recipient_NPI'].isin(npi_list)]

    for idx, row in individual_payment.iterrows():
        cur_nct = row['ClinicalTrials_Gov_Identifier']
        cur_npi = row['Covered_Recipient_NPI']
        
        if cur_nct not in data_dict:
            data_dict[cur_nct] = [{}, {}, {}]
        if cur_npi not in data_dict[cur_nct][0]:
            data_dict[cur_nct][0][cur_npi] = row['Total_Amount_of_Payment_USDollars']
        else:
            data_dict[cur_nct][0][cur_npi] += row['Total_Amount_of_Payment_USDollars']

    for idx, row in institution_payment.iterrows():
        cur_nct = row['ClinicalTrials_Gov_Identifier']
        cur_hos = row['Teaching_Hospital_ID']
        
        if cur_nct not in data_dict:
            data_dict[cur_nct] = [{}, {}, {}]
        if cur_hos not in data_dict[cur_nct][1]:
            data_dict[cur_nct][1][cur_hos] = {}
            for i in range(1, 6):
                if str(row['Principal_Investigator_' + str(i) + '_NPI']) == 'nan':
                    continue
                if row['Principal_Investigator_' + str(i) + '_NPI'] in data_dict[cur_nct][1][cur_hos]:
                    data_dict[cur_nct][1][cur_hos][row['Principal_Investigator_' + str(i) + '_NPI']] += row['Total_Amount_of_Payment_USDollars']
                else:
                    data_dict[cur_nct][1][cur_hos][row['Principal_Investigator_' + str(i) +
                    '_NPI']] = row['Total_Amount_of_Payment_USDollars']

    for idx, row in entity_payment.iterrows():
        cur_nct = row['ClinicalTrials_Gov_Identifier']
        cur_ent = row['Noncovered_Recipient_Entity_Name']
        
        if cur_nct not in data_dict:
            data_dict[cur_nct] = [{}, {}, {}]
        if cur_ent not in data_dict[cur_nct][2]:
            data_dict[cur_nct][2][cur_ent] = {}
            for i in range(1, 6):
                if str(row['Principal_Investigator_' + str(i) + '_NPI']) == 'nan':
                    continue
                if row['Principal_Investigator_' + str(i) + '_NPI'] in data_dict[cur_nct][2][cur_ent]:
                    data_dict[cur_nct][2][cur_ent][row['Principal_Investigator_' + str(i) + '_NPI']] += row['Total_Amount_of_Payment_USDollars']
                else:
                    data_dict[cur_nct][2][cur_ent][row['Principal_Investigator_' + str(i) + '_NPI']] = row['Total_Amount_of_Payment_USDollars']

C:\Users\Junyi\AppData\Local\Temp\ipykernel_19992\2857075477.py:20: DtypeWarning: Columns (2,5,8,9,10,11,18,19,20,26,32,33,34,35,36,50,51,67,68,75,77,97,101,104,105,106,107,108,109,110,111,112,113,116,122,128,133,136,137,138,139,140,141,142,143,144,145,148,154,160,165,168,169,170,171,172,173,174,175,176,177,180,186,192,193,219,225,231,237,238,239) have mixed types. Specify dtype option on import or set low_memory=False.
  payment = pd.read_csv('D:/Data/payment/Research/OP_DTL_RSRCH_PGYR' + str(i) + '_P01202023.csv', dtype={'Covered_Recipient_NPI':str, 'Principal_Investigator_1_NPI': str, 'Principal_Investigator_2_NPI': str, 'Principal_Investigator_3_NPI': str, 'Principal_Investigator_4_NPI': str, 'Principal_Investigator_5_NPI': str, 'ClinicalTrials_Gov_Identifier': str})
C:\Users\Junyi\AppData\Local\Temp\ipykernel_19992\2857075477.py:20: DtypeWarning: Columns (2,5,8,9,10,11,18,19,20,26,32,33,34,35,36,50,51,67,68,69,72,73,74,75,76,77,78,79,80,81,84,90,96,97,98,101,104,105,106,107,108,10

In [26]:
pickle.dump(data_dict, open('../data/payment_dict.pkl', 'wb'))

In [6]:
payment_dict = pickle.load(open('../data/payment_dict.pkl', 'rb'))
trial_enroll = pickle.load(open('../data/trial_enroll.pkl', 'rb'))
trial2npi = pickle.load(open('../data/trial2npi.pkl', 'rb'))

In [19]:
trial2cost = {}
for trial in payment_dict:
    total_cost = 0
    for npi in payment_dict[trial][0]:
        total_cost += payment_dict[trial][0][npi]
    for hos in payment_dict[trial][1]:
        for npi in payment_dict[trial][1][hos]:
            total_cost += payment_dict[trial][1][hos][npi]
    for ent in payment_dict[trial][2]:
        for npi in payment_dict[trial][2][ent]:
            total_cost += payment_dict[trial][2][ent][npi]
    trial2cost[trial] = total_cost
    
trial_enroll_clean = {}
for trial in trial_enroll:
    if trial_enroll[trial] == '0' or trial_enroll[trial] == None:
        continue
    trial_enroll_clean[trial] = int(trial_enroll[trial])
    
npi2cost = {}
for trial in payment_dict:
    if trial not in trial_enroll_clean or int(trial_enroll_clean[trial]) == 0:
        continue
    for npi in payment_dict[trial][0]:
        if npi not in npi2cost:
            npi2cost[npi] = []
        npi2cost[npi].append(payment_dict[trial][0][npi] / int(trial_enroll_clean[trial]))
    for hos in payment_dict[trial][1]:
        for npi in payment_dict[trial][1][hos]:
            if npi not in npi2cost:
                npi2cost[npi] = []
            npi2cost[npi].append(payment_dict[trial][1][hos][npi] / int(trial_enroll_clean[trial]))
    for ent in payment_dict[trial][2]:
        for npi in payment_dict[trial][2][ent]:
            if npi not in npi2cost:
                npi2cost[npi] = []
            npi2cost[npi].append(payment_dict[trial][2][ent][npi] / int(trial_enroll_clean[trial]))
for npi in npi2cost:
    npi2cost[npi] = np.mean(npi2cost[npi])
    
npi2cost_all = {}
for trial in payment_dict:
    if trial not in trial_enroll_clean or int(trial_enroll_clean[trial]) == 0:
        continue
    for npi in payment_dict[trial][0]:
        if npi not in npi2cost_all:
            npi2cost_all[npi] = []
        npi2cost_all[npi].append(payment_dict[trial][0][npi])
    # for hos in payment_dict[trial][1]:
    #     for npi in payment_dict[trial][1][hos]:
    #         if npi not in npi2cost_all:
    #             npi2cost_all[npi] = []
    #         npi2cost_all[npi].append(payment_dict[trial][1][hos][npi])
    # for ent in payment_dict[trial][2]:
    #     for npi in payment_dict[trial][2][ent]:
    #         if npi not in npi2cost_all:
    #             npi2cost_all[npi] = []
    
    #         npi2cost_all[npi].append(payment_dict[trial][2][ent][npi])
for npi in npi2cost_all:
    cur_cost = npi2cost_all[npi]
    # Mean of 20% - 80% quantile
    cur_cost = sorted(cur_cost)
    top_idx = int(len(cur_cost) * 0.2)
    bot_idx = int(len(cur_cost) * 0.8)
    if top_idx == bot_idx:
        npi2cost_all[npi] = np.mean(cur_cost)
    else:
        npi2cost_all[npi] = np.mean(cur_cost[top_idx:bot_idx])

In [20]:
pickle.dump(npi2cost, open('../data/npi2cost.pkl', 'wb'))
pickle.dump(trial2cost, open('../data/trial2cost.pkl', 'wb'))
pickle.dump(trial_enroll_clean, open('../data/trial_enroll_clean.pkl', 'wb'))

In [21]:
pickle.dump(npi2cost_all, open('../data/npi2cost_all.pkl', 'wb'))

In [13]:
trial2ratio = {}
for trial in payment_dict:
    if trial not in trial_enroll_clean or int(trial_enroll_clean[trial]) == 0:
        continue
    trial2ratio[trial] = {}
    for npi in payment_dict[trial][0]:
        trial2ratio[trial][npi] = payment_dict[trial][0][npi] / trial2cost[trial]
    for hos in payment_dict[trial][1]:
        for npi in payment_dict[trial][1][hos]:
            trial2ratio[trial][npi] = payment_dict[trial][1][hos][npi]  / trial2cost[trial]
    for ent in payment_dict[trial][2]:
        for npi in payment_dict[trial][2][ent]:
            trial2ratio[trial][npi] = payment_dict[trial][2][ent][npi]  / trial2cost[trial]

In [15]:
pickle.dump(trial2ratio, open('../data/trial2ratio.pkl', 'wb'))